In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time


def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False):
    if debugflag:
        model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(2,1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        return model
        
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        layer_type = layer_config[0]

        if layer_type == 'dense':
            neurons, activation = layer_config[1], layer_config[2]
            if len(layer_config) > 3 and layer_config[3] == 'input':
                input_shape = layer_config[4]
                model.add(layers.Dense(neurons, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(layers.Dense(neurons, activation=activation))
        elif layer_type == 'dropout':
            rate = layer_config[1]
            model.add(layers.Dropout(rate))
        elif layer_type == 'activation':
            activation = layer_config[1]
            model.add(layers.Activation(activation))
        elif layer_type == 'conv2d':
            filters, kernel_size, strides, activation = layer_config[1], layer_config[2], layer_config[3], layer_config[4]
            model.add(layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, activation=activation))
        elif layer_type == 'maxpool2d':
            pool_size = layer_config[1]
            model.add(layers.MaxPool2D(pool_size=pool_size))
        elif layer_type == 'rnn':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences))
        elif layer_type == 'lstm':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.LSTM(units, return_sequences=return_sequences))

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, y_train, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2", total_epoch=50,ea=False,eamonitor="val_loss"):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor=eamonitor, patience=7, min_delta=0.001, verbose=1)


    start_time = time.time()
    if ea:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback,early_stopping])
    else:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2",threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test,threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    #threshold = 0.5  # Define the threshold
    interpredictions=predictions.flatten()
    correct_predictions = np.sum(np.abs(true_labels - interpredictions) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, interpredictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, interpredictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), interpredictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }


In [2]:
# Load dataset
dataset_name = "addition_dataset_2"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 91458680.0000 - mae: 4641.7783 - val_loss: 0.1382 - val_mae: 0.3093
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1400 - mae: 0.3119 - val_loss: 0.1385 - val_mae: 0.3138
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1415 - mae: 0.3134 - val_loss: 0.1383 - val_mae: 0.3073
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1398 - mae: 0.3107 - val_loss: 0.1377 - val_mae: 0.3139
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1395 - mae: 0.3107 - val_loss: 0.1352 - val_mae: 0.3090
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2147 - mae: 0.3450 - val_loss: 0.1349 - val_mae: 0.3102
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1364 - mae: 0.3083 - val_loss: 0.1366 - val_mae: 0.3150
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1377 - mae: 0.3103 - val_loss: 0.1414 - val_mae: 0.3085
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms

In [3]:
# Load dataset
dataset_name = "doubling_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 163441296.0000 - mae: 7511.2637 - val_loss: 0.1703 - val_mae: 0.3443
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1730 - mae: 0.3463 - val_loss: 0.1700 - val_mae: 0.3447
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1758 - mae: 0.3497 - val_loss: 0.1697 - val_mae: 0.3432
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1733 - mae: 0.3466 - val_loss: 0.1691 - val_mae: 0.3434
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 996us/step - loss: 0.1724 - mae: 0.3450 - val_loss: 0.1689 - val_mae: 0.3439
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1718 - mae: 0.3453 - val_loss: 0.1691 - val_mae: 0.3466
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1716 - mae: 0.3457 - val_loss: 0.1677 - val_mae: 0.3449
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1710 - mae: 0.3447 - val_loss: 0.1741 - val_mae: 0.3422
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - 

In [4]:
# Load dataset
dataset_name = "substraction_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8795022.0000 - mae: 1060.8656 - val_loss: 0.0047 - val_mae: 0.0574
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step - loss: 0.0048 - mae: 0.0573 - val_loss: 0.0047 - val_mae: 0.0574
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0048 - mae: 0.0578 - val_loss: 0.0047 - val_mae: 0.0568
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 997us/step - loss: 0.0048 - mae: 0.0578 - val_loss: 0.0047 - val_mae: 0.0577
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - loss: 0.0047 - mae: 0.0573 - val_loss: 0.0046 - val_mae: 0.0569
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - loss: 0.0047 - mae: 0.0570 - val_loss: 0.0046 - val_mae: 0.0561
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0047 - mae: 0.0574 - val_loss: 0.0045 - val_mae: 0.0569
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0046 - mae: 0.0564 - val_loss: 0.0045 - val_mae: 0.0553
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/ste

In [16]:
# Load dataset
dataset_name = "simple_sin_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 343403.5938 - mae: 207.8793 - val_loss: 0.5087 - val_mae: 0.6439
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5068 - mae: 0.6363 - val_loss: 0.5151 - val_mae: 0.6452
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5142 - mae: 0.6409 - val_loss: 0.5375 - val_mae: 0.6529
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5200 - mae: 0.6421 - val_loss: 0.5073 - val_mae: 0.6435
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5356 - mae: 0.6451 - val_loss: 0.5302 - val_mae: 0.6506
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5421 - mae: 0.6497 - val_loss: 0.5315 - val_mae: 0.6496
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5741 - mae: 0.6585 - val_loss: 0.5080 - val_mae: 0.6434
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 973us/step - loss: 0.5952 - mae: 0.6670 - val_loss: 0.9078 - val_mae: 0.7808
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss

In [19]:
# Load dataset
dataset_name = "simple_cos_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 49648.0469 - mae: 74.8407 - val_loss: 0.4953 - val_mae: 0.6339
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5161 - mae: 0.6420 - val_loss: 0.4952 - val_mae: 0.6339
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5499 - mae: 0.6568 - val_loss: 0.4960 - val_mae: 0.6342
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5416 - mae: 0.6520 - val_loss: 0.5767 - val_mae: 0.6580
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5467 - mae: 0.6491 - val_loss: 0.4965 - val_mae: 0.6343
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5616 - mae: 0.6548 - val_loss: 0.4994 - val_mae: 0.6347
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7003 - mae: 0.7061 - val_loss: 0.5751 - val_mae: 0.6598
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6621 - mae: 0.6934 - val_loss: 0.6984 - val_mae: 0.7015
Epoch 8: early stopping
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/

In [7]:
# Load dataset
dataset_name = "multiplication_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=1500,ea=True,eamonitor="mae")

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/1500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 30215706960199680.0000 - mae: 128070336.0000 - val_loss: 4739719212564480.0000 - val_mae: 47643608.0000
Epoch 2/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2672290265300992.0000 - mae: 41189996.0000 - val_loss: 1997759343230976.0000 - val_mae: 39665716.0000
Epoch 3/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2039165646536704.0000 - mae: 39968628.0000 - val_loss: 1998104953880576.0000 - val_mae: 39722260.0000
Epoch 4/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1973160320696320.0000 - mae: 39253952.0000 - val_loss: 2007960964300800.0000 - val_mae: 40056712.0000
Epoch 5/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2002068269170688.0000 - mae: 39638792.0000 - val_loss: 1998907978547200.0000 - val_mae: 39787564.0000
Epoch 6/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2008579573809152.0000 - mae: 39697340.0000 - val_loss: 2002283554406400.0000 - val_mae: 39915268.0000
Epoch 7/1500
500/500 ━━━━━━

In [8]:
# Load dataset
dataset_name = "exponent_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv',nrows=16)
d_set['result']=d_set['result'].astype('int')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 972ms/step - loss: 546036578368094208.0000 - mae: 516242720.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 7/50
1/1 ━━━━━

In [21]:
# Load dataset
dataset_name = "natural_log_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=1500,ea=True,eamonitor="mae")

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/1500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 103303.7500 - mae: 106.4455 - val_loss: 13.9249 - val_mae: 3.2059
Epoch 2/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 14.0681 - mae: 3.2254 - val_loss: 13.5540 - val_mae: 3.1882
Epoch 3/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.5988 - mae: 3.1694 - val_loss: 12.8779 - val_mae: 3.0972
Epoch 4/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.2397 - mae: 3.1309 - val_loss: 12.4179 - val_mae: 2.9951
Epoch 5/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 12.8028 - mae: 3.0843 - val_loss: 12.1711 - val_mae: 2.9490
Epoch 6/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 12.7686 - mae: 3.0517 - val_loss: 10.0734 - val_mae: 2.7654
Epoch 7/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 10.9437 - mae: 2.8558 - val_loss: 8.6625 - val_mae: 2.5377
Epoch 8/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 10.2934 - mae: 2.7880 - val_loss: 8.2727 - val_mae: 2.4426
Epoch 9/1500
500/500 ━━━━━━━━━━━━

In [22]:
# Load dataset
dataset_name = "integration_with_cos_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=1500,ea=True,eamonitor="mae")

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/1500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1097.3455 - mae: 22.4542 - val_loss: 980.0704 - val_mae: 21.8617
Epoch 2/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 808.8227 - mae: 19.2860 - val_loss: 49.3797 - val_mae: 4.4705
Epoch 3/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 31.2329 - mae: 3.1581 - val_loss: 24.9691 - val_mae: 2.4206
Epoch 4/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.3472 - mae: 2.3172 - val_loss: 22.1369 - val_mae: 2.1846
Epoch 5/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.0436 - mae: 2.2373 - val_loss: 20.3808 - val_mae: 2.2172
Epoch 6/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 24.3902 - mae: 2.3762 - val_loss: 24.0295 - val_mae: 2.2230
Epoch 7/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.0300 - mae: 2.2455 - val_loss: 19.8614 - val_mae: 2.1552
Epoch 8/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 22.6940 - mae: 2.2476 - val_loss: 19.9909 - val_mae: 2.0615
Epoch 9/1500
500/500 ━━━━━━━━━

In [23]:
# Load dataset
dataset_name = "integration_with_sin_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=1500,ea=True,eamonitor="mae")

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 667.8182 - mae: 18.7644 - val_loss: 590.3171 - val_mae: 17.4336
Epoch 2/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 551.2769 - mae: 16.1618 - val_loss: 425.7437 - val_mae: 11.7985
Epoch 3/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 412.6239 - mae: 10.9820 - val_loss: 385.2934 - val_mae: 9.8459
Epoch 4/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 379.0697 - mae: 9.6671 - val_loss: 331.5040 - val_mae: 8.7874
Epoch 5/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 296.0459 - mae: 8.4165 - val_loss: 156.0409 - val_mae: 5.7509
Epoch 6/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 111.7848 - mae: 4.8896 - val_loss: 19.4805 - val_mae: 1.8940
Epoch 7/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 16.7178 - mae: 1.8250 - val_loss: 3.1313 - val_mae: 0.7589
Epoch 8/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.8809 - mae: 0.8092 - val_loss: 0.6011 - val_mae: 0.3927
Epoch 9/150

In [24]:
# Load dataset
dataset_name = "integration_with_tan_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=1500,ea=True,eamonitor="mae")

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/1500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 39331124.0000 - mae: 199.1649 - val_loss: 11124388.0000 - val_mae: 211.5812
Epoch 2/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 22051388.0000 - mae: 168.5208 - val_loss: 11124748.0000 - val_mae: 211.6656
Epoch 3/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13317541.0000 - mae: 147.4144 - val_loss: 11125366.0000 - val_mae: 211.9076
Epoch 4/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 25502118.0000 - mae: 180.4657 - val_loss: 11126321.0000 - val_mae: 212.4852
Epoch 5/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 56985080.0000 - mae: 228.0427 - val_loss: 11126742.0000 - val_mae: 212.7924
Epoch 6/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 32967042.0000 - mae: 204.7207 - val_loss: 11120795.0000 - val_mae: 213.4782
Epoch 7/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 78524424.0000 - mae: 278.7537 - val_loss: 11128220.0000 - val_mae: 214.1275
Epoch 8/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1m